### Capstone 2- Bengali Grapheme Classification Project

This workbook is the first attempt at experimenting with a CNN for this project. I've followed this [starter code] (https://www.kaggle.com/kaushal2896/bengali-graphemes-starter-eda-multi-output-cnn)for ideas. 

This is also the first attempt at using MLFlow for tracking deep learning experiments

In [1]:
#importing necessary packages
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import sklearn
from collections import defaultdict
import mlflow

Although my virtual environment uses tensorflow-gpu, I want to doubly make sure that keras uses my GPU

In [2]:
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

Using TensorFlow backend.


['/job:localhost/replica:0/task:0/device:GPU:0']

In [3]:
# using custom scripts for creating model and loading data
import sys
# insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, './model/')

In [4]:
from model_creator import model_create
from data_loader import data_loader

In [5]:
# read the csv files
filenames = ['train','test','class_map','class_map_corrected','train_multi_diacritics','sample_submission']
df_dict = defaultdict()

for file in filenames:
    df_dict[file]=pd.read_csv('./data/{}.csv'.format(file))

In [6]:
# using mlflow autologger to track models, artifacts and parameters
import mlflow.keras
mlflow.keras.autolog()

In [8]:
#creating mlflow experiment
mlflow.create_experiment(name="conv_kernel_size_6_pool_size_3_default_arch")

'1'

In [9]:
#this version of the model create function creates a 3 layer alternating conv-maxpool layers followed by dropout
#two dense layers with a dropout in the middle and three output layers

#to-do batch normalization, add optimizer and initializer as parameters
model = model_create(conv_kernel_size=6,pool_size=3)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.



./model/model_creator.py:60: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(pool_size=(3, 3), data_format="channels_last")`
  model = MaxPool2D(pool_size=(pool_size, pool_size), dim_ordering="tf")(model)


In [10]:
#view summary of model created
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_layer (InputLayer)        (None, 137, 236, 1)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 137, 236, 64) 2368        input_layer[0][0]                
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 45, 78, 64)   0           conv2d_1[0][0]                   
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 45, 78, 64)   147520      max_pooling2d_1[0][0]            
____________________________________________________________________________________________

In [11]:
#specify hyper-parameters
batch_size = 256
epochs = 50

In [12]:
#note - this implementation did not save experiment under the experiment name and instead saved it in the default name

#to do - investigate why
with mlflow.start_run():
    for i in range(4):
        #iterate through all 4 training parquet files 
        print('Reading parquet file #{}'.format(i+1))
        print('---------------------------------------')
        print('Transforming data for parquet file #{}'.format(i+1))
        print('---------------------------------------')
        x_train, x_test, y_train_root, y_test_root, y_train_consonant, y_test_consonant, y_train_vowel, y_test_vowel=data_loader('./data/train_image_data_{}.parquet'.format(i),df_dict['train'])
        print('Training model on parquet file #{}'.format(i+1))
        print('---------------------------------------')
        history=model.fit(x=x_train, y={"output_root":y_train_root,
                                    "output_vowel":y_train_vowel,"output_consonant":y_train_consonant},
                      epochs=epochs,batch_size=batch_size,
                      validation_data=(x_test,{"output_root":y_test_root,"output_vowel":y_test_vowel,
                                               "output_consonant":y_test_consonant}))
        print('Deleting variables after training')
        del x_train, x_test, y_train_root, y_test_root, y_train_consonant, y_test_consonant, y_train_vowel, y_test_vowel


Reading parquet file #1
---------------------------------------
Transforming data for parquet file #1
---------------------------------------
Training model on parquet file #1
---------------------------------------


/home/jayanth/anaconda3/envs/tf1_env/lib/python3.6/site-packages/mlflow/utils/autologging_utils.py:60: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  all_param_names, _, _, all_default_values = inspect.getargspec(fn)  # pylint: disable=W1505



Train on 45189 samples, validate on 5021 samples
Epoch 1/50
45189/45189 [==============================] - 70s 2ms/step - loss: 9.2900 - output_root_loss: 5.1647 - output_vowel_loss: 2.4848 - output_consonant_loss: 1.6369 - output_root_accuracy: 0.0259 - output_vowel_accuracy: 0.1964 - output_consonant_accuracy: 0.6197 - val_loss: 8.0627 - val_output_root_loss: 4.7187 - val_output_vowel_loss: 2.1073 - val_output_consonant_loss: 1.2376 - val_output_root_accuracy: 0.0313 - val_output_vowel_accuracy: 0.2243 - val_output_consonant_accuracy: 0.6208
Epoch 2/50
45189/45189 [==============================] - 61s 1ms/step - loss: 7.8907 - output_root_loss: 4.6921 - output_vowel_loss: 2.0247 - output_consonant_loss: 1.1737 - output_root_accuracy: 0.0291 - output_vowel_accuracy: 0.2515 - output_consonant_accuracy: 0.6263 - val_loss: 7.7126 - val_output_root_loss: 4.6511 - val_output_vowel_loss: 1.8656 - val_output_consonant_loss: 1.1967 - val_output_root_accuracy: 0.0337 - val_output_vowel_accur

Epoch 17/50
45189/45189 [==============================] - 56s 1ms/step - loss: 1.4442 - output_root_loss: 1.0068 - output_vowel_loss: 0.2258 - output_consonant_loss: 0.2112 - output_root_accuracy: 0.6936 - output_vowel_accuracy: 0.9203 - output_consonant_accuracy: 0.9241 - val_loss: 1.8241 - val_output_root_loss: 1.3039 - val_output_vowel_loss: 0.2454 - val_output_consonant_loss: 0.2723 - val_output_root_accuracy: 0.6256 - val_output_vowel_accuracy: 0.9187 - val_output_consonant_accuracy: 0.9064
Epoch 18/50
45189/45189 [==============================] - 56s 1ms/step - loss: 1.3562 - output_root_loss: 0.9454 - output_vowel_loss: 0.2098 - output_consonant_loss: 0.2008 - output_root_accuracy: 0.7089 - output_vowel_accuracy: 0.9260 - output_consonant_accuracy: 0.9275 - val_loss: 1.8894 - val_output_root_loss: 1.3617 - val_output_vowel_loss: 0.2643 - val_output_consonant_loss: 0.2631 - val_output_root_accuracy: 0.6136 - val_output_vowel_accuracy: 0.9122 - val_output_consonant_accuracy: 0.9

45189/45189 [==============================] - 56s 1ms/step - loss: 0.8254 - output_root_loss: 0.5613 - output_vowel_loss: 0.1344 - output_consonant_loss: 0.1300 - output_root_accuracy: 0.8239 - output_vowel_accuracy: 0.9541 - output_consonant_accuracy: 0.9530 - val_loss: 1.8329 - val_output_root_loss: 1.3132 - val_output_vowel_loss: 0.2615 - val_output_consonant_loss: 0.2565 - val_output_root_accuracy: 0.6461 - val_output_vowel_accuracy: 0.9158 - val_output_consonant_accuracy: 0.9181
Epoch 34/50
45189/45189 [==============================] - 56s 1ms/step - loss: 0.7911 - output_root_loss: 0.5354 - output_vowel_loss: 0.1328 - output_consonant_loss: 0.1236 - output_root_accuracy: 0.8313 - output_vowel_accuracy: 0.9544 - output_consonant_accuracy: 0.9564 - val_loss: 1.7933 - val_output_root_loss: 1.2826 - val_output_vowel_loss: 0.2480 - val_output_consonant_loss: 0.2598 - val_output_root_accuracy: 0.6680 - val_output_vowel_accuracy: 0.9217 - val_output_consonant_accuracy: 0.9213
Epoch 35

...

/home/jayanth/anaconda3/envs/tf1_env/lib/python3.6/site-packages/mlflow/utils/autologging_utils.py:60: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  all_param_names, _, _, all_default_values = inspect.getargspec(fn)  # pylint: disable=W1505


45189/45189 [==============================] - 58s 1ms/step - loss: 2.6207 - output_root_loss: 1.7938 - output_vowel_loss: 0.4548 - output_consonant_loss: 0.3710 - output_root_accuracy: 0.5332 - output_vowel_accuracy: 0.8533 - output_consonant_accuracy: 0.8794 - val_loss: 1.6372 - val_output_root_loss: 1.1534 - val_output_vowel_loss: 0.2441 - val_output_consonant_loss: 0.2380 - val_output_root_accuracy: 0.6716 - val_output_vowel_accuracy: 0.9225 - val_output_consonant_accuracy: 0.9195
Epoch 2/50
45189/45189 [==============================] - 58s 1ms/step - loss: 2.0414 - output_root_loss: 1.4206 - output_vowel_loss: 0.3306 - output_consonant_loss: 0.2901 - output_root_accuracy: 0.6063 - output_vowel_accuracy: 0.8925 - output_consonant_accuracy: 0.9042 - val_loss: 1.4563 - val_output_root_loss: 1.0306 - val_output_vowel_loss: 0.2165 - val_output_consonant_loss: 0.2066 - val_output_root_accuracy: 0.7070 - val_output_vowel_accuracy: 0.9331 - val_output_consonant_accuracy: 0.9343
Epoch 3/5

45189/45189 [==============================] - 59s 1ms/step - loss: 1.0026 - output_root_loss: 0.6980 - output_vowel_loss: 0.1541 - output_consonant_loss: 0.1500 - output_root_accuracy: 0.7834 - output_vowel_accuracy: 0.9457 - output_consonant_accuracy: 0.9470 - val_loss: 1.5499 - val_output_root_loss: 1.1089 - val_output_vowel_loss: 0.2233 - val_output_consonant_loss: 0.2107 - val_output_root_accuracy: 0.6897 - val_output_vowel_accuracy: 0.9287 - val_output_consonant_accuracy: 0.9315
Epoch 18/50
45189/45189 [==============================] - 59s 1ms/step - loss: 0.9753 - output_root_loss: 0.6721 - output_vowel_loss: 0.1528 - output_consonant_loss: 0.1512 - output_root_accuracy: 0.7909 - output_vowel_accuracy: 0.9471 - output_consonant_accuracy: 0.9467 - val_loss: 1.5148 - val_output_root_loss: 1.0899 - val_output_vowel_loss: 0.2094 - val_output_consonant_loss: 0.2095 - val_output_root_accuracy: 0.6875 - val_output_vowel_accuracy: 0.9301 - val_output_consonant_accuracy: 0.9299
Epoch 19

45189/45189 [==============================] - 61s 1ms/step - loss: 0.5328 - output_root_loss: 0.3615 - output_vowel_loss: 0.0870 - output_consonant_loss: 0.0847 - output_root_accuracy: 0.8833 - output_vowel_accuracy: 0.9699 - output_consonant_accuracy: 0.9702 - val_loss: 1.7955 - val_output_root_loss: 1.2887 - val_output_vowel_loss: 0.2555 - val_output_consonant_loss: 0.2435 - val_output_root_accuracy: 0.6945 - val_output_vowel_accuracy: 0.9311 - val_output_consonant_accuracy: 0.9305
Epoch 34/50
45189/45189 [==============================] - 61s 1ms/step - loss: 0.5208 - output_root_loss: 0.3540 - output_vowel_loss: 0.0841 - output_consonant_loss: 0.0826 - output_root_accuracy: 0.8859 - output_vowel_accuracy: 0.9709 - output_consonant_accuracy: 0.9702 - val_loss: 1.8039 - val_output_root_loss: 1.2975 - val_output_vowel_loss: 0.2541 - val_output_consonant_loss: 0.2420 - val_output_root_accuracy: 0.6945 - val_output_vowel_accuracy: 0.9293 - val_output_consonant_accuracy: 0.9297
Epoch 35

45189/45189 [==============================] - 59s 1ms/step - loss: 0.4405 - output_root_loss: 0.2984 - output_vowel_loss: 0.0737 - output_consonant_loss: 0.0688 - output_root_accuracy: 0.9060 - output_vowel_accuracy: 0.9737 - output_consonant_accuracy: 0.9755 - val_loss: 1.8673 - val_output_root_loss: 1.3405 - val_output_vowel_loss: 0.2557 - val_output_consonant_loss: 0.2631 - val_output_root_accuracy: 0.6801 - val_output_vowel_accuracy: 0.9253 - val_output_consonant_accuracy: 0.9245
Epoch 50/50
45189/45189 [==============================] - 59s 1ms/step - loss: 0.4113 - output_root_loss: 0.2774 - output_vowel_loss: 0.0674 - output_consonant_loss: 0.0661 - output_root_accuracy: 0.9131 - output_vowel_accuracy: 0.9767 - output_consonant_accuracy: 0.9763 - val_loss: 1.7656 - val_output_root_loss: 1.2749 - val_output_vowel_loss: 0.2448 - val_output_consonant_loss: 0.2403 - val_output_root_accuracy: 0.6897 - val_output_vowel_accuracy: 0.9287 - val_output_consonant_accuracy: 0.9329
Deleting

In [45]:
# getting predictions on test set
# this code is borrowed from the started code mentioned at the top but modified for my implementation

preds_dict = {
    'grapheme_root': [],
    'vowel_diacritic': [],
    'consonant_diacritic': []
}

components = ['consonant_diacritic', 'grapheme_root', 'vowel_diacritic']
target=[] # model predictions placeholder
row_id=[] # row_id place holder
for i in range(4):
    df_test_img = pd.read_parquet('./data/test_image_data_{}.parquet'.format(i)) 
    df_test_img.set_index('image_id', inplace=True)


    X_test = df_test_img.values.reshape(-1, 137, 236, 1)
    
    preds = model.predict(X_test)

    for i, p in enumerate(preds_dict):
        preds_dict[p] = np.argmax(preds[i], axis=1)

    for k,id in enumerate(df_test_img.index.values):  
        for i,comp in enumerate(components):
            id_sample=id+'_'+comp
            row_id.append(id_sample)
            target.append(preds_dict[comp][k])
    del df_test_img
    del X_test


df_sample = pd.DataFrame(
    {
        'row_id': row_id,
        'target':target
    },
    columns = ['row_id','target'] 
)
df_sample.to_csv('submission.csv',index=False)
df_sample


,row_id,target
0,Test_0_consonant_diacritic,0
1,Test_0_grapheme_root,3
2,Test_0_vowel_diacritic,0
3,Test_1_consonant_diacritic,0
4,Test_1_grapheme_root,93
5,Test_1_vowel_diacritic,2
6,Test_2_consonant_diacritic,0
7,Test_2_grapheme_root,19
8,Test_2_vowel_diacritic,0
9,Test_3_consonant_diacritic,0
